# Basic development and testing of the polymer inverse design case study

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import time 
import pandas as pd 
import matplotlib.pyplot as plt 
plt.style.use(['nature', 'science'])

from gpt3forchem.data import get_polymer_data